In [22]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import requests
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pickle
import warnings

warnings.filterwarnings('ignore')

selected_crypto = None
data = None

# Load the trained model
with open('rf_model.pkl', 'rb') as f:
    model = pickle.load(f)
    

# Function to fetch and display data
def fetch_data():
    global selected_crypto, data
    for widget in prediction_frame.winfo_children():
        widget.destroy()
    crypto = crypto_var.get()
    if not crypto:
        messagebox.showerror("Error", "Please select a cryptocurrency.")
        return
    selected_crypto = crypto  # Assign the value to the global variable

    # Fetch data
    df = fetch_crypto_and_fng_data(selected_crypto)
    # add technical indicators
    add_Technical_Indicators(df)
    df.dropna(inplace=True)
    # set info data
    set_info_data(df)
    # Save the DataFrame to a CSV file
    file_path = 'data.csv'
    df.to_csv(file_path, index=True)
    #load data from a file
    data = load_data_file()
    # display price chart
    display_price_volume_chart(data, selected_crypto, plot_frame)


def fetch_crypto_and_fng_data(crypto):
    # Fetch historical daily data from Yahoo Finance
    
    crypto_df = yf.Ticker(crypto)
    crypto_df = crypto_df.history(period="max", interval="1d")
    crypto_df = pd.DataFrame(crypto_df, columns=['Open', 'High', 'Low', 'Close', 'Volume'])
    crypto_df.reset_index(inplace=True)
    crypto_df.rename(columns={'Date': 'timestamp'}, inplace=True)
    crypto_df['timestamp'] = pd.to_datetime(crypto_df['timestamp'])
    crypto_df.set_index('timestamp', inplace=True)
    crypto_df.index = crypto_df.index.tz_localize(None)  # Make the index timezone-naive

    # Ensure the data has a consistent daily frequency by resampling
    crypto_df = crypto_df.resample('D').ffill()

    # Fetch Fear and Greed Index historical data from api.alternative.me
    response = requests.get("https://api.alternative.me/fng/?limit=0&format=json").json()['data']
    fear_greed_df = pd.DataFrame(response, columns=['timestamp', 'value'])
    fear_greed_df['timestamp'] = pd.to_numeric(fear_greed_df['timestamp'])  # Explicitly cast to numeric
    fear_greed_df['timestamp'] = pd.to_datetime(fear_greed_df['timestamp'], unit='s')
    fear_greed_df.set_index('timestamp', inplace=True)
    fear_greed_df.rename(columns={'value': 'fear_greed_index'}, inplace=True)
    fear_greed_df.index = fear_greed_df.index.tz_localize(None)  # Make the index timezone-naive

    # Ensure the Fear and Greed Index data has a consistent daily frequency by resampling
    fear_greed_df = fear_greed_df.resample('D').ffill()

    # Merge the two datasets based on their dates
    df = crypto_df.merge(fear_greed_df, how='left', left_index=True, right_index=True)
    df.dropna(inplace=True)
    
    # Apply simple exponential smoothing to the close price
    smooth = SimpleExpSmoothing(df['Close']).fit(smoothing_level=0.5, optimized=False)
    df['Smoothed'] = smooth.fittedvalues

    return df
    
def set_info_data(df):
    ticker.set(crypto_var.get())
    date.set(f"{df.index[-1].strftime('%Y-%m-%d')}")
    open_price.set(f"{df['Open'][-1]:.2f}")
    high_price.set(f"{df['High'][-1]:.2f}")
    low_price.set(f"{df['Low'][-1]:.2f}")
    current_price.set(f"{df['Close'][-1]:.2f}")
    volume.set(f"{df['Volume'][-1]:,}")
    f_n_g.set(f"{df['fear_greed_index'][-1]}")
    

def load_data_file():
    # load the data file
    file_path = 'data.csv'
    data = pd.read_csv(file_path, parse_dates=['timestamp'], index_col='timestamp')
    data = data.tail(90)
    return data


def add_Technical_Indicators(df):
    
    # add indicators to the table using pandas ta library
    # Calculate the 200-day EMA
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()
    # calculate macd
    df.ta.macd(close='Close', fast=12, slow=26, signal=9, append=True)
    #rename columns 
    df.rename(columns={'MACD_12_26_9': 'macd_line', 'MACDs_12_26_9': 'macd_signal', 'MACDh_12_26_9': 'macd_hist'}, inplace=True)
    # Calculate RSI with default settings (14 periods)
    df['RSI_7'] = ta.rsi(df['Close'], length=7)
    # Calculate OBV using pandas-ta
    df['OBV'] = ta.obv(df['Close'], df['Volume'])
    #comodity chanel index
    df['CCI'] = ta.cci(df['High'], df['Low'], df['Close'], length=20)  # Commodity Channel Index
    # Add Momentum Indicator (MOM)
    df['Momentum'] = ta.mom(df['Close'], length=10)
    # Adding Awesome_Oscillator
    df['Awesome_Oscillator'] = ta.ao(df['High'], df['Low'])  
    
    return df

def display_price_volume_chart(data, selected_crypto, plot_frame):
    # Clear previous plots
    for widget in plot_frame.winfo_children():
        widget.destroy()
    
    # Plot price, and volume
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(4, 2), sharex=True, gridspec_kw={'height_ratios': [4, 1]})
    

    # Plot price
    ax1.plot(data.index, data['Close'], label='Close Price')
    ax1.set_ylabel('Price')
    ax1.set_title(f'{selected_crypto} 90 days Price')
    ax1.legend(loc='upper left')

    # Plot volume 
    ax2.bar(data.index, data['Volume'], label='Volume', color='blue')
    ax2.set_ylabel('Volume')
    ax2.legend(loc='upper left')

    # Set x-axis label on the bottom subplot
    ax2.set_xlabel('Date')

    # Adjust the space between subplots
    plt.subplots_adjust(hspace=0.1)

    # Display the plot
    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    # close the figure
    plt.close(fig)

    
def display_price_EMA200_volume(data, selected_crypto, plot_frame):
    # Clear previous plots
    for widget in plot_frame.winfo_children():
        widget.destroy()

    # Plot price, EMA 200, and volume
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(4, 2), sharex=True, gridspec_kw={'height_ratios': [4, 1]})

    # Plot price and EMA 200
    ax1.plot(data.index, data['Close'], label='Close Price')
    ax1.plot(data.index, data['EMA_200'], label='200-day EMA', color='orange')
    ax1.set_ylabel('Price')
    ax1.set_title(f'{selected_crypto} 90 days Price and 200-day EMA')
    ax1.legend(loc='upper left')

    # Plot volume 
    ax2.bar(data.index, data['Volume'], label='Volume', color='blue')
    ax2.set_ylabel('Volume')
    ax2.legend(loc='upper left')

    # Set x-axis label on the bottom subplot
    ax2.set_xlabel('Date')

    # Adjust the space between subplots
    plt.subplots_adjust(hspace=0.1)

    # Display the plot
    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    # close the figure
    plt.close(fig)

def display_macd(data, selected_crypto, plot_frame):
    # Clear previous plots
    for widget in plot_frame.winfo_children():
        widget.destroy()

    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(4, 2), sharex=True, gridspec_kw={'height_ratios': [4, 1]})

    # Plot the price
    ax1.plot(data.index, data['Close'], label='Close Price')
    ax1.set_ylabel('Price')
    ax1.set_title(f'{selected_crypto} 90 days Price and MACD')
    ax1.legend(loc='upper left')

    # Plot MACD
    ax2.plot(data.index, data['macd_line'], label='MACD Line', color='blue')
    ax2.plot(data.index, data['macd_signal'], label='MACD Signal', color='red')
    ax2.bar(data.index, data['macd_hist'], label='MACD Histogram', color='gray')
    ax2.set_ylabel('MACD')
    ax2.legend(loc='upper left')

    # Set x-axis label on the bottom subplot
    ax2.set_xlabel('Date')

    # Adjust the space between subplots
    plt.subplots_adjust(hspace=0.1)
    
    # Display the plot
    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    # close the figure
    plt.close(fig)

def display_rsi_close_chart(data, selected_crypto, plot_frame):
    # Clear previous plots
    for widget in plot_frame.winfo_children():
        widget.destroy()

    # Plot closing price and RSI
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(4, 2), sharex=True, gridspec_kw={'height_ratios': [4, 1]})

    # Plot closing price
    ax1.plot(data.index, data['Close'], label='Close Price')
    ax1.set_ylabel('Price')
    ax1.set_title(f'{selected_crypto} 90 days Price and RSI')
    ax1.legend(loc='upper left')

    # Plot RSI
    ax2.plot(data.index, data['RSI_7'], label='RSI', color='red')
    ax2.axhline(y=70, color='gray', linestyle='--', linewidth=1)  # Overbought line
    ax2.axhline(y=30, color='gray', linestyle='--', linewidth=1)  # Oversold line
    ax2.set_ylabel('RSI')
    ax2.set_xlabel('Date')
    ax2.legend(loc='upper left')

    # Adjust the space between subplots
    plt.subplots_adjust(hspace=0.1)

    # Display the plot
    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    # close the figure
    plt.close(fig)

def display_obv_close_chart(data, selected_crypto, plot_frame):
    # Clear previous plots
    for widget in plot_frame.winfo_children():
        widget.destroy()

    # Plot closing price and OBV
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(4, 2), sharex=True, gridspec_kw={'height_ratios': [4, 1]})

    # Plot closing price
    ax1.plot(data.index, data['Close'], label='Close Price')
    ax1.set_ylabel('Price')
    ax1.set_title(f'{selected_crypto} 90 days Price and OBV')
    ax1.legend(loc='upper left')

    # Plot OBV
    ax2.plot(data.index, data['OBV'], label='OBV', color='orange')
    ax2.set_ylabel('OBV')
    ax2.set_xlabel('Date')
    ax2.legend(loc='upper left')

    # Adjust the space between subplots
    plt.subplots_adjust(hspace=0.1)

    # Display the plot
    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    # close the figure
    plt.close(fig)
    
def display_cci_close_chart(data, selected_crypto, plot_frame):
    # Clear previous plots
    for widget in plot_frame.winfo_children():
        widget.destroy()

    # Plot closing price and CCI
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(4, 2), sharex=True, gridspec_kw={'height_ratios': [4, 1]})

    # Plot closing price
    ax1.plot(data.index, data['Close'], label='Close Price')
    ax1.set_ylabel('Price')
    ax1.set_title(f'{selected_crypto} 90 days Price and CCI')
    ax1.legend(loc='upper left')

    # Plot CCI
    ax2.plot(data.index, data['CCI'], label='CCI', color='green')
    ax2.axhline(y=100, color='gray', linestyle='--', linewidth=1)  # Overbought line
    ax2.axhline(y=-100, color='gray', linestyle='--', linewidth=1)  # Oversold line
    ax2.set_ylabel('CCI')
    ax2.set_xlabel('Date')
    ax2.legend(loc='upper left')

    # Adjust the space between subplots
    plt.subplots_adjust(hspace=0.1)

    # Display the plot
    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    # close the figure
    plt.close(fig)

def display_awesome_oscillator_close_chart(data, selected_crypto, plot_frame):
    # Clear previous plots
    for widget in plot_frame.winfo_children():
        widget.destroy()

    # Plot closing price and Awesome Oscillator
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(4, 2), sharex=True, gridspec_kw={'height_ratios': [4, 1]})

    # Plot closing price
    ax1.plot(data.index, data['Close'], label='Close Price')
    ax1.set_ylabel('Price')
    ax1.set_title(f'{selected_crypto} 90 days Price and Awesome Oscillator')
    ax1.legend(loc='upper left')

    # Plot Awesome Oscillator
    ax2.plot(data.index, data['Awesome_Oscillator'], label='Awesome Oscillator', color='green')
    ax2.set_ylabel('Awesome Oscillator')
    ax2.set_xlabel('Date')
    ax2.legend(loc='upper left')

    # Adjust the space between subplots
    plt.subplots_adjust(hspace=0.1)

    # Display the plot
    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    # close the figure
    plt.close(fig)
    
# Function to confirm and quit application
def quit_application():
    answer = messagebox.askyesno("Quit", "Are you sure you want to quit?")
    if answer:
        root.destroy()  

# function to handle indicators buttons clicks
def ta_button_click(indicator):
    if (indicator == 'Price & Volume'):
        display_price_volume_chart(data, selected_crypto, plot_frame)
    if (indicator == 'EMA 200'):
        display_price_EMA200_volume(data, selected_crypto, plot_frame)
        # disply MACD chart
    if (indicator == 'MACD_12_26_9'):
        display_macd(data, selected_crypto, plot_frame)
    if (indicator == 'RSI_7'):
        display_rsi_close_chart(data, selected_crypto, plot_frame)
    if (indicator == 'OBV'):
        display_obv_close_chart(data, selected_crypto, plot_frame)
    if (indicator == 'CCI'):
        display_cci_close_chart(data, selected_crypto, plot_frame)
    if (indicator == 'AO'):
        display_awesome_oscillator_close_chart(data, selected_crypto, plot_frame)


#Function to handle forecast button click
def predict_button_click():
    for widget in prediction_frame.winfo_children():
        widget.destroy()
    data = load_data_file()
    data = data.tail(30)  # Use the last 30 days of data
    predict, proba = make_predictions(data, model, selected_crypto)
    prediction_text = f"Prediction for the {selected_crypto} price for next 3 days is to go {'Up' if predict[0] == 1 else 'Down'}."
    tk.Label(prediction_frame, text=prediction_text, bg="white", font=("Helvetica", 14)).pack(expand=True)

def make_predictions(data, model, crypto):
    # Ensure the data is preprocessed in the same way as during training
    features = ['EMA_200', 'macd_line', 'RSI_7', 'OBV', 'Momentum', 'CCI', 'Awesome_Oscillator', 'fear_greed_index']
    last_30_days = data[features].values
    last_30_days = last_30_days.reshape(1, -1)  # Reshape for prediction
    predictions = model.predict(last_30_days)
    predictions_prob = model.predict_proba(last_30_days)
    return predictions, predictions_prob

# Create main window
root = tk.Tk()
root.title("Crypto Price Predictor")

# Make the application window maximized
root.state('zoomed')
# Bind the close window button to the quit confirmation
root.protocol("WM_DELETE_WINDOW", quit_application)
root.geometry("1200x800")
root.configure(bg="#195770")



# Create a top frame for the selection and fetch button and prediction display frames
top_frame = tk.Frame(root, bg="#195770", height=20)
top_frame.pack(fill=tk.X, pady=10, padx=10)

# Create a frame for the selection and button
selection_frame = tk.Frame(top_frame, bg="#195770", height=20)
selection_frame.pack(fill=tk.X, pady=10, padx=10, side=tk.LEFT)

# create frame for predictions display
prediction_frame = tk.Frame(top_frame, bg="#195770", height=20)
prediction_frame.pack(fill=tk.X, pady=10, padx=10, side=tk.LEFT)
   
# Create main display frame
main_frame = tk.Frame(root, bg="white", bd=2, relief=tk.SUNKEN)
main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=(0, 0))

# Create a frame for the info
info_frame = tk.Frame(main_frame, bg="#195770", width=200)
info_frame.pack(fill=tk.Y, padx=10, pady=10, side=tk.LEFT)

# Create a frame for the plot
plot_frame = tk.Frame(main_frame, bg="#195770", bd=2, width=600)
plot_frame.pack(fill=tk.BOTH, expand=True, padx=0, pady=10, side=tk.LEFT)

# Create a frame for the technical indicators
TA_frame = tk.Frame(main_frame, bg="#195770", width=150)
TA_frame.pack(fill=tk.BOTH, padx=10, pady=10, side=tk.LEFT)

# create technical indicators buttons
indicators = ['Price & Volume', 'EMA 200', 'MACD_12_26_9', 'RSI_7', 'OBV', 'CCI', 'AO']
for indicator in indicators:
    button = tk.Button(TA_frame, text=indicator, command=lambda ind=indicator: ta_button_click(ind), bg="lightblue", fg="black")
    button.pack(fill=tk.X, pady=10, padx=5)

# create forecast button
forecast_button = tk.Button(TA_frame, text="Predict", command=predict_button_click, bg="pink", fg="black")
forecast_button.pack(fill=tk.X, pady=20, padx=5)

# Create labels for displaying information
ticker = tk.StringVar()
date = tk.StringVar()
open_price = tk.StringVar()
high_price = tk.StringVar()
low_price = tk.StringVar()
current_price = tk.StringVar()
volume = tk.StringVar()
f_n_g = tk.StringVar()
# Set the font size
font = ("Helvetica", 14)

def initialize_labels(info_frame, font, ticker, date, open_price, high_price, low_price, current_price, volume, f_n_g):
    # Initialize labels with default text
    
    tk.Label(info_frame, textvariable=ticker, bg="#195770", fg="white", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, text="Date:", bg="#195770", fg="#EEC44F", font=font).pack(side=tk.TOP, padx=1, pady=(10, 0))
    tk.Label(info_frame, textvariable=date, bg="#195770", fg="white", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, text="Open:", bg="#195770", fg="#EEC44F", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, textvariable=open_price, bg="#195770", fg="white", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, text="High:", bg="#195770", fg="#EEC44F", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, textvariable=high_price, bg="#195770", fg="white", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, text="Low:", bg="#195770", fg="#EEC44F", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, textvariable=low_price, bg="#195770", fg="white", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, text="Current Price:", bg="#195770", fg="#EEC44F", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, textvariable=current_price, bg="#195770", fg="white", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, text="Volume:", bg="#195770", fg="#EEC44F", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, textvariable=volume, bg="#195770", fg="white", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, text="Fear & Greed:", bg="#195770", fg="#EEC44F", font=font).pack(side=tk.TOP, padx=1)
    tk.Label(info_frame, textvariable=f_n_g, bg="#195770", fg="white", font=font).pack(side=tk.TOP, padx=1)

# Example usage
initialize_labels(info_frame, font, ticker, date, open_price, high_price, low_price, current_price, volume, f_n_g)

# List of top 10 cryptocurrencies
top_10_cryptos = ["BTC-USD", "ETH-USD", "BNB-USD", "ADA-USD", "SOL-USD", "XRP-USD", "DOT-USD", "DOGE-USD", "LTC-USD", "AVAX-USD"]

# Create a dropdown menu for selecting cryptocurrency
crypto_var = tk.StringVar()
crypto_dropdown = ttk.Combobox(selection_frame, textvariable=crypto_var, values=top_10_cryptos)
crypto_dropdown.set("Select crypto")
crypto_dropdown.pack(side=tk.LEFT, padx=(100, 0), pady=10)


# Create a button to fetch data
fetch_button = tk.Button(selection_frame, text="Fetch", command=fetch_data, bg="green", fg="white")
fetch_button.pack(side=tk.LEFT, padx=10, pady=10)

# Fetch data for the first cryptocurrency in the list on startup
#fetch_data()

# Run the application
root.mainloop()
